In [1]:
from haystack import Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.preprocessors import DocumentSplitter
from haystack.components.embedders import (
    SentenceTransformersDocumentEmbedder,
    SentenceTransformersTextEmbedder,
)
from haystack.components.retrievers import InMemoryEmbeddingRetriever
from haystack.components.writers import DocumentWriter
from haystack.components.builders import PromptBuilder
from haystack.components.generators import HuggingFaceLocalGenerator
from haystack.dataclasses import Document


c:\Users\alira\anaconda3\envs\sorbonne\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
document_store = InMemoryDocumentStore()


In [3]:
documents = [
    Document(content="Artificial intelligence is transforming healthcare."),
    Document(content="Machine learning is a subset of artificial intelligence."),
    Document(content="الذكاء الاصطناعي يغير مجال الرعاية الصحية."),
    Document(content="تعلم الآلة هو جزء من الذكاء الاصطناعي.")
]


In [4]:
splitter = DocumentSplitter(
    split_by="sentence",
    split_length=5,
    split_overlap=1
)


In [5]:
doc_embedder = SentenceTransformersDocumentEmbedder(
    model="intfloat/multilingual-e5-small",
    prefix="passage: "
)

doc_embedder.warm_up()


Loading weights: 100%|██████████| 199/199 [00:00<00:00, 897.96it/s, Materializing param=pooler.dense.weight]                               
BertModel LOAD REPORT from: intfloat/multilingual-e5-small
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [6]:
ingestion_pipeline = Pipeline()

ingestion_pipeline.add_component("splitter", splitter)
ingestion_pipeline.add_component("embedder", doc_embedder)
ingestion_pipeline.add_component("writer", DocumentWriter(document_store))

ingestion_pipeline.connect("splitter", "embedder")
ingestion_pipeline.connect("embedder", "writer")


🚅 Components
  - splitter: DocumentSplitter
  - embedder: SentenceTransformersDocumentEmbedder
  - writer: DocumentWriter
🛤️ Connections
  - splitter.documents -> embedder.documents (list[Document])
  - embedder.documents -> writer.documents (list[Document])

In [7]:
ingestion_pipeline.run({
    "splitter": {"documents": documents}
})


Batches: 100%|██████████| 1/1 [00:00<00:00,  1.65it/s]


{'writer': {'documents_written': 4}}

In [8]:
stored_docs = document_store.filter_documents()
print("Documents stored:", len(stored_docs))
print("Embedding dimension:", len(stored_docs[0].embedding))


Documents stored: 4
Embedding dimension: 384


In [9]:
query_embedder = SentenceTransformersTextEmbedder(
    model="intfloat/multilingual-e5-small",
    prefix="query: "
)

query_embedder.warm_up()


In [10]:
retriever = InMemoryEmbeddingRetriever(document_store)


In [11]:
prompt_template = """
You are a helpful assistant.
Use ONLY the following documents to answer the question.
If the answer is not present, say "I don't know".

Documents:
{% for doc in documents %}
{{ doc.content }}
{% endfor %}

Question: {{ query }}
Answer:
"""


In [12]:
prompt_builder = PromptBuilder(template=prompt_template)


PromptBuilder has 2 prompt variables, but `required_variables` is not set. By default, all prompt variables are treated as optional, which may lead to unintended behavior in multi-branch pipelines. To avoid unexpected execution, ensure that variables intended to be required are explicitly set in `required_variables`.


In [13]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")


True
NVIDIA GeForce RTX 4070 Laptop GPU


In [14]:
from haystack.components.generators import OpenAIGenerator
from haystack.utils import Secret

llm = OpenAIGenerator(
    api_base_url="http://localhost:8000/v1",
    api_key=Secret.from_token("dummy"),  # 👈 wrap it
    model="Qwen/Qwen2.5-1.5B-Instruct"
)


In [15]:
rag_pipeline = Pipeline()

rag_pipeline.add_component("query_embedder", query_embedder)
rag_pipeline.add_component("retriever", retriever)
rag_pipeline.add_component("prompt_builder", prompt_builder)
rag_pipeline.add_component("llm", llm)

rag_pipeline.connect("query_embedder", "retriever")
rag_pipeline.connect("retriever", "prompt_builder.documents")
rag_pipeline.connect("prompt_builder", "llm")


🚅 Components
  - query_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - llm: OpenAIGenerator
🛤️ Connections
  - query_embedder.embedding -> retriever.query_embedding (list[float])
  - retriever.documents -> prompt_builder.documents (list[Document])
  - prompt_builder.prompt -> llm.prompt (str)

In [16]:
result = rag_pipeline.run({
    "query_embedder": {"text": "ما هو الذكاء الاصطناعي؟"},
    "prompt_builder": {"query": "ما هو الذكاء الاصطناعي؟"}
})

print(result["llm"]["replies"][0])


Batches: 100%|██████████| 1/1 [00:00<00:00, 39.99it/s]


PipelineRuntimeError: The following component failed to run:
Component name: 'llm'
Component type: 'OpenAIGenerator'
Error: Connection error.

In [ ]:
import time

start = time.time()
_ = llm.run("Hello")
print("LLM only time:", time.time() - start)


LLM only time: 0.7554011344909668


In [ ]:
result = rag_pipeline.run({
    "query_embedder": {"text": "ما هو الذكاء الاصطناعي؟"},
    "prompt_builder": {"query": "ما هو الذكاء الاصطناعي؟"}
})

print(result["llm"]["replies"][0])
